<a href="https://colab.research.google.com/github/jeffufpost/stochastic_SEIR_model/blob/master/COLAB_stochastic_SEIR_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

from scipy.stats import gamma
from scipy.stats import poisson

In [0]:
# Some variables for the next things
p = 50000
days = 100

# Epidemiology of SEIR model and initial population array

### Introduction:
We want to model an epidemic with 4 individual states that are {S, E, I, R}.
Instead of using the standard differential equation approach, we want to model the changes from one state to the next in a stochastic manner that depend on external factors as described below.

Comparison with COVID-19: 

Some assumptions must be made, notably that a recovered person is either dead or fully and indefinitly immunised.

### State S - susceptible:
An individual is susceptible to the disease when they are not immune to the disease. In an SEIR model, the individual in state S can be exposed to the disease, and go into the state E. The probability of this happening depends on three key variables:
* beta : the number of contacts the individual has per day
* I/p : the probability of a contact being with an infectious individual
* r : the probability of a contact with an infectious individual leading to infection.

### State E - exposed:
Once an individual is exposed to the disease (changing from S as described above), they become carriers for the disease but it takes a certain amount of time before they become infectious. While the incubation period denotes the time between exposure and development of disease (appearance of symptoms), it is possible for an individual to become infectious before this.

In toher words, the time for an individual between E and I can be shorter, longer, or the same as the incubation time, depending on the indivudal and the virus.

This model will use a  [gamma distribution](https://mathworld.wolfram.com/GammaDistribution.html) for the time it takes an indivudal to go from E to I state.



### State I - infectious:
A certain amount of time after exposure, an individual will become infectious. These indivudals are responsible for spreading the disease by a formula containing the three points described in state S.

An infectious person will at some point either recover or die from the disease. The infectious period can be shorter, longer, or the same as the time it takes for them to recover or die. We denote the change of states from I to R as the time for an individual to stop being infectious.

This model will also use a  [gamma distribution](https://mathworld.wolfram.com/GammaDistribution.html) for this process.



### State R - recovered:
Following the infectious period, the person will stop becoming infectious (by either recovering or dying) and the will enter the state R.

# Probability distributions of time from E to I and from I to R

## E --> I

### Gamma distribution

In [0]:
g1 = pd.DataFrame(gamma.rvs(1.9, loc=1.7, scale=2.2, size=100000), columns=['g'])

In [0]:
g1.describe()

,g
count,100000.000000
mean,5.889100
std,3.052764
min,1.704603
25%,3.654943
50%,5.166566
75%,7.355245
max,34.175622


In [0]:
# Literature suggests 95% hae symptoms within 14 days of exposure
g1.quantile(q=0.95)

g    11.858259
Name: 0.95, dtype: float64

In [0]:
fig = px.histogram(g1, x='g', nbins=100)
fig.show()

### Poisson distribution

Rather difficult to model the incubation time as a poisson distribution.

The high mu needed to match the 95th percentile at 14 days results in a higher mean/median than the 5 days suggested in the literature.


In [0]:
p1 = pd.DataFrame(poisson.rvs(mu=3, loc=2, size=100000), columns=['p'])

In [0]:
p1.describe()

,p
count,100000.000000
mean,5.001010
std,1.734906
min,2.000000
25%,4.000000
50%,5.000000
75%,6.000000
max,15.000000


In [0]:
p1.quantile(q=0.95)

p    8.0
Name: 0.95, dtype: float64

In [0]:
fig = px.histogram(p1, x='p', nbins=100)
fig.show()

## I --> R

### Gamma distribution

In [0]:
g2 = pd.DataFrame(gamma.rvs(7, loc=4, scale=1, size=10000), columns=['g'])

In [0]:
g2.describe()

,g
count,10000.000000
mean,10.957484
std,2.645870
min,4.790719
25%,9.044327
50%,10.570069
75%,12.529870
max,25.364863


In [0]:
g2.quantile(q=0.95)

g    15.857547
Name: 0.95, dtype: float64

In [0]:
fig = px.histogram(pd.DataFrame(gamma.rvs(7, loc=4, scale=1, size=10000), columns=['g']), x='g', nbins=100)
fig.show()

### Poisson distribution

Similarly as above, rather difficult to model the recovery time as a poisson distribution.

In [0]:
p2 = pd.DataFrame(poisson.rvs(mu=5, loc=4, size=100000), columns=['p'])

In [0]:
p2.describe()

,p
count,100000.000000
mean,8.992790
std,2.233313
min,4.000000
25%,7.000000
50%,9.000000
75%,10.000000
max,20.000000


In [0]:
p2.quantile(q=0.95)

p    13.0
Name: 0.95, dtype: float64

In [0]:
fig = px.histogram(p2, x='p', nbins=100)
fig.show()

## Comparison side by side

In [0]:
x=np.arange(50)
g1=gamma.cdf(x,1.5,loc=2.5,scale=3)
g2=gamma.cdf(x,7,loc=4,scale=1)

In [0]:
fig = go.Figure(data=[
    go.Scatter(name='E->I', x=np.arange(20), y=g1),    
    go.Scatter(name='I->R', x=np.arange(20), y=g2)
])

fig.show()

## Other - masking for selection of individuals

In [0]:
rand = np.random.random(size=(p, days))

In [0]:
rand

array([[0.58177614, 0.96903614, 0.72770699, ..., 0.21103833, 0.97156806,
        0.32904789],
       [0.758561  , 0.37994789, 0.96578968, ..., 0.33394067, 0.64617575,
        0.56442073],
       [0.90030203, 0.5308046 , 0.76314654, ..., 0.90763405, 0.20568519,
        0.25330504],
       ...,
       [0.01331328, 0.53703974, 0.43289731, ..., 0.71493954, 0.42622493,
        0.62482589],
       [0.2725718 , 0.94590902, 0.8050957 , ..., 0.76314466, 0.18844407,
        0.59210427],
       [0.85855426, 0.46639669, 0.04173146, ..., 0.56927063, 0.55598155,
        0.39972972]])

In [0]:
g1=gamma.cdf(np.arange(70),1.5,loc=2.5,scale=3)

In [0]:
g1

array([0.        , 0.        , 0.        , 0.04635783, 0.19874804,
       0.35563019, 0.49383478, 0.60837482, 0.70021941, 0.77235289,
       0.82820286, 0.87099694, 0.90352768, 0.92810223, 0.94657278,
       0.96039764, 0.97070911, 0.97837716, 0.98406482, 0.98827412,
       0.99138322, 0.99367569, 0.99536339, 0.99660415, 0.99751519,
       0.99818335, 0.99867289, 0.99903121, 0.99929326, 0.99948475,
       0.99962457, 0.9997266 , 0.999801  , 0.99985523, 0.99989472,
       0.99992348, 0.9999444 , 0.99995962, 0.99997068, 0.99997872,
       0.99998456, 0.9999888 , 0.99999188, 0.99999411, 0.99999573,
       0.99999691, 0.99999776, 0.99999838, 0.99999883, 0.99999915,
       0.99999938, 0.99999955, 0.99999968, 0.99999977, 0.99999983,
       0.99999988, 0.99999991, 0.99999994, 0.99999995, 0.99999997,
       0.99999998, 0.99999998, 0.99999999, 0.99999999, 0.99999999,
       1.        , 1.        , 1.        , 1.        , 1.        ])

In [0]:
j = np.arange(15)
df=make_df(p,500)
l = list(map(lambda x: rand[:,x] < g1[x-df.Day], j))

In [0]:
for i in range(0,len(l)):
  print(l[i].sum()/500000)

0.0
0.0
0.0
0.004682
0.020054
0.035584
0.049586
0.060822
0.069812
0.07685
0.083028
0.087212
0.090452
0.092874
0.094726


In [0]:
maskg1 = (rand[:,:] < g1[:])
maskg2 = (rand[:,:] < g2[:])

ValueError: ignored

In [0]:
for i in range(0,len(l)):
  print(maskg1[:,i].sum()/500000)


NameError: ignored

# SEIR model

In [0]:
# Epidemiology and context setting
p = 500000
beta = 5
r = 0.3
days = 100 # number of days on which to run the epidemic

In [0]:
%%timeit
data = {'State': np.full((p,1), 'S').T[0], 'Day': np.full((p,1), 0).T[0]}
df = pd.DataFrame(data, columns=['State', 'Day'])

10 loops, best of 3: 24.8 ms per loop


In [0]:
def make_df(p,num_I):
  df = pd.DataFrame(np.full((p,1), 'S').T[0], columns=['State'])
  df['Day'] = 0
  df.loc[np.random.randint(1,p,num_I),'State'] = 'I'
  return df

In [0]:
df = make_df(5000,100)

In [0]:
S=np.array([],dtype=int)
E=np.array([],dtype=int)
I=np.array([],dtype=int)
R=np.array([],dtype=int)
Cases=np.array([],dtype=int)

In [0]:
r=0.3
beta=5
beta2=0.5
p=5000
num_I=100
days=40

In [0]:
g1=gamma.cdf(np.arange(days),2,loc=2.5,scale=2.5)
g2=gamma.cdf(np.arange(days),7,loc=4,scale=1)

In [0]:
g1.shape

(40,)

In [0]:
g2.shape

(40,)

In [0]:
rand = np.random.random(size=(p,days))

In [0]:
rand.shape

(50000, 40)

In [0]:
df.Day = np.random.randint(0,20,p)

In [0]:
df.Day

0        16
1        14
2         8
3        12
4         2
         ..
49995     3
49996     8
49997     8
49998    17
49999     5
Name: Day, Length: 50000, dtype: int64

In [0]:
maskg1 = (rand[:,:] < g1[:])
maskg2 = (rand[:,:] < g2[:])

We have to understand whether it is faster to have a boolean mask that depends on current and df.Day, or faster to have a simple comparison (argmax below)


In [0]:
#The code below will get the first day where the mask becomes true for each individual in p
maskday1 = maskg1.argmax(1)

In [0]:
#The code below will get the first day where the mask becomes true for each individual in p
maskday2 = maskg2.argmax(1)

In [0]:
maskday1

array([7, 3, 5, ..., 5, 8, 6])

In [0]:
df.Day.T

0         9
1        18
2        19
3         5
4         3
         ..
49995     3
49996     8
49997     6
49998    14
49999    18
Name: Day, Length: 50000, dtype: int64

In [0]:
maskg1.shape

(50000, 40)

In [0]:
len(df.Day < maskday2)

50000

In [0]:
df.Day.values

array([ 9, 18, 19, ...,  6, 14, 18])

In [0]:
maskg1.shape

(50000, 40)

In [0]:
df.iloc[df.State[df.State=='S'].index].shape

(49800, 2)

In [0]:
df.loc[(df.Day > maskday1)] = ['E', 0]

In [0]:
df.loc[(df.Day >= maskday2) & (df.State == 'I')]

,State,Day
421,I,17
660,I,15
779,I,16
1438,I,15
1445,I,10
...,...,...
49005,I,19
49139,I,17
49180,I,15
49607,I,13


In [0]:
df.loc[(df.State == 'S') & (rand[:,0] < r*beta*len(np.where(df.State=='I')[0])/p)]

,State,Day
3,S,0
7,S,0
12,S,0
29,S,0
104,S,0
...,...,...
4907,S,0
4915,S,0
4916,S,0
4933,S,0


In [0]:
df.iloc[df.State[df.State=='S'].index].where(maskg1[df.State[df.State=='S'].index,20-df.Day[df.State=='S']])

ValueError: ignored

In [0]:
df1 = df.State =='S'

In [0]:
df1.shape

(50000,)

In [0]:
df.Day.shape

(50000,)

In [0]:
maskg1[:,df.Day]

In [0]:
df2.shape

In [0]:
maskg1[df.State[df.State=='S'].index,20-df.Day[df.State=='S']].shape

(49800,)

In [0]:
pd.Series(maskg1[df.State[df.State=='S'].index,0-df.Day[df.State=='S']])

RangeIndex(start=0, stop=49800, step=1)

In [0]:
df.loc[(pd.Series(maskg1[:,0-df.Day]).values)]

In [0]:
maskg1[:,0-df.Day]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [0]:
type(maskg1[df.State[df.State=='S'].index,20-df.Day[df.State=='S']])

(49800,)

In [0]:
df.loc[df.State=='S',:].loc[(maskg1[df.State[df.State=='S'].index,20-df.Day[df.State=='S']]),:].shape

(49400, 2)

In [0]:
  S=np.array([],dtype=int)
  E=np.array([],dtype=int)
  I=np.array([],dtype=int)
  R=np.array([],dtype=int)
  Cases=np.array([],dtype=int)

In [0]:
df.loc[(df.State == 'I') & (df.Day >= maskday2)]

,State,Day


In [0]:
df.loc[(df.State == 'E') & (df.Day >= maskday1)]

,State,Day


In [0]:
df.loc[(df.State == 'S') & (rand[:,0] < r*beta*len(np.where(df.State=='I')[0])/p)]

,State,Day
3,S,0
7,S,0
12,S,0
29,S,0
104,S,0
...,...,...
4907,S,0
4915,S,0
4916,S,0
4933,S,0


In [0]:
Cases=np.append(Cases,len(df.loc[(df.State == 'E') & (df.Day >= maskday1)]))

S=np.append(S,len(np.where(df.State=='S')[0]))
E=np.append(E,len(np.where(df.State=='E')[0]))
I=np.append(I,len(np.where(df.State=='I')[0]))
R=np.append(R,len(np.where(df.State=='R')[0]))

In [0]:
E

array([0])

In [0]:
len(df.State[df.State=='S'].index)

49800

In [0]:
maskg1[df.State[df.State=='S'].index,6-df.Day[df.State=='S']]

array([ True, False,  True, ..., False, False,  True])

In [0]:
mask1 = maskg1[df.State[df.State=='S'].index,20-df.Day[df.State=='S']]

In [0]:
maskg1.shape

(50000, 40)

In [0]:
df.iloc[maskg1[df.State[df.State=='S'].index,20-df.Day[df.State=='S']]]

In [0]:
maskg1[1,0-df.Day]

array([False, False, False, ..., False, False, False])

In [0]:
df.loc[(df.State=='S') & (maskg1[0-df.Day])]

In [0]:
20-df.Day

In [0]:
(df.State[df.State=='S']) & (maskg1[df.State[df.State=='S'].index,0])

In [0]:
len(df.State[(df.State=='S') & (maskg1[:,0-df.Day])])

In [0]:
len(df[(df.State=='E') & (maskg1[:,0-df.Day])])

In [0]:
def SEIR_model(r, beta, beta2, p, num_I, days, g1=None, g2=None):

  df = make_df(p,num_I)

  S=np.array([],dtype=int)
  E=np.array([],dtype=int)
  I=np.array([],dtype=int)
  R=np.array([],dtype=int)
  Cases=np.array([],dtype=int)

  Epd=np.array([],dtype=int)
  Ipd=np.array([],dtype=int)
  Rpd=np.array([],dtype=int)
  Casespd=np.array([],dtype=int)

  if g1 is None:
    g1=gamma.cdf(np.arange(days),2,loc=2.5,scale=2.5)
  else:
    g1 = g1
    
  if g2 is None:
    g2=gamma.cdf(np.arange(days),7,loc=4,scale=1)
  else:
    g2 = g2

  b=beta

  rand = np.random.random(size=(p,days))

  maskg1 = (rand[:,:] < g1[:])
  maskg2 = (rand[:,:] < g2[:])

  maskday1 = maskg1.argmax(1)
  maskday2 = maskg2.argmax(1)

  for j in range(0,days-1):

    Rpd=np.append(Rpd,len(df.loc[(df.State == 'I') & (j-df.Day >= maskday2)]))
    df.loc[(df.State == 'I') & (j-df.Day >= maskday2)] = ['R', j]

    Ipd=np.append(Ipd,len(df.loc[(df.State == 'E') & (j-df.Day >= maskday1)]))
    df.loc[(df.State == 'E') & (j-df.Day >= maskday1)] = ['I', j]

    Epd=np.append(Epd,len(df.loc[(df.State == 'S') & (rand[:,j] < r*b*len(np.where(df.State=='I')[0])/p)]))
    df.loc[(df.State == 'S') & (rand[:,j] < r*b*len(np.where(df.State=='I')[0])/p)] = ['E', j]
                            
    #df[(df.State=='S') & (np.random.random(size=p) < r*b*len(np.where(df.State=='I')[0])/p)] = ['E', j]
      
    #Cases=np.append(Cases,len(df.loc[(df.State == 'E') & (df.Day >= maskday1)]))

    #df[(df.State == 'I') & (rand[:,j] < g2[j-df.Day])] = ['R', j]   
    #df[(df.State == 'E') & (rand[:,j] < g1[j-df.Day])] = ['I', j]

    #df[(df.State=='I') & (maskg2[:,j-df.Day])] = ['R', j]

    #df[(df.State=='E') & (maskg1[:,j-df.Day])] = ['I', j] 

#    df[(df.State == 'I') & (rand[:,j] < gamma.cdf(j-df.Day,7,loc=4,scale=1))] = ['R', j]
    
#    df[(df.State == 'E') & (rand[:,j] < gamma.cdf(j-df.Day,2,loc=2.5,scale=2.5))] = ['I', j]

    #df[(df.State == 'S') & (rand[:,j] < r*b*len(np.where(df.State == 'I')[0])/p)] = ['E', j]      

#        df[(df.State=='S') & (rand[:,j] < r*b*len(np.where(df.State=='I')[0])/p)] = ['E', j]
#        df[(df.State=='E') & maskg1[:,j]] = ['I', j]
#        df[(df.State=='I') & maskg2[:,j]] = ['R', j]
    S=np.append(S,len(np.where(df.State=='S')[0]))
    E=np.append(E,len(np.where(df.State=='E')[0]))
    I=np.append(I,len(np.where(df.State=='I')[0]))
    R=np.append(R,len(np.where(df.State=='R')[0]))

    if Ipd.cumsum()[-1] > (p*0.1): b = beta2
    elif Ipd[-1] < 150: b = beta

  Ipd[0]+=num_I
  
  return S,Epd,Ipd,Rpd

In [0]:
%%timeit
SEIR_model(0.1,3,0.7,5000,1000,100)

1 loop, best of 3: 907 ms per loop


In [0]:
%%time
values = np.arange(0.1,0.5,0.1)
l = list(map(lambda x: SEIR_model(x,3,0.7,50000,200,100), values))

CPU times: user 18.5 s, sys: 144 ms, total: 18.7 s
Wall time: 18.7 s


In [0]:
np.arange(0.1,0.5,0.1)

array([0.1, 0.2, 0.3, 0.4])

In [0]:
l[3][4].cumsum()

array([], dtype=int64)

In [0]:
l[0][2][30:]

array([456, 546, 604, 675, 668, 651, 565, 417, 346, 269, 268, 287, 268,
       262, 265, 288, 263, 270, 275, 219, 218, 192, 177, 149, 128, 153,
       141, 138, 134, 107,  99, 116, 116,  72,  91,  79,  74,  92,  74,
        54,  75,  49,  53,  51,  53,  42,  33,  26,  49,  34,  29,  31,
        33,  27,  25,  28,  20,  17,  22,  21,  12,  21,  13,  20,  12,
        12,  17,  19,  14])

In [0]:
fig = go.Figure(data=[
    go.Scatter(name='S for r={}'.format(values[0]), x=np.arange(len(l[0][0])), y=l[0][0]),
    go.Scatter(name='E for r={}'.format(values[0]), x=np.arange(len(l[0][1])), y=l[0][1]),
    go.Scatter(name='I for r={}'.format(values[0]), x=np.arange(len(l[0][2])), y=l[0][2]),
    go.Scatter(name='R for r={}'.format(values[0]), x=np.arange(len(l[0][3])), y=l[0][3]),                  
    #go.Scatter(name='Cases for r={}'.format(values[0]), x=np.arange(len(l[0][4])), y=l[0][4].cumsum())
])

In [0]:
fig2 = go.Figure(data=[
    go.Scatter(name='S for r={}'.format(values[0]), x=np.arange(len(l[0][0])), y=l[0][0]),
    go.Bar(name='Epd for r={}'.format(values[0]), x=np.arange(len(l[0][1])), y=l[0][1]),
    go.Bar(name='Ipd for r={}'.format(values[0]), x=np.arange(len(l[0][2])), y=l[0][2]),
    go.Bar(name='Rpd for r={}'.format(values[0]), x=np.arange(len(l[0][3])), y=l[0][3]),                  
    go.Scatter(name='Cases for r={}'.format(values[0]), x=np.arange(len(l[0][3])), y=l[0][3].cumsum())
])

In [0]:
for i in range(1,len(values)):
    fig.add_trace(go.Scatter(name='S for r={}'.format(values[i]), x=np.arange(len(l[i][0])), y=l[i][0]))
    fig.add_trace(go.Scatter(name='E for r={}'.format(values[i]), x=np.arange(len(l[i][1])), y=l[i][1]))
    fig.add_trace(go.Scatter(name='I for r={}'.format(values[i]), x=np.arange(len(l[i][2])), y=l[i][2]))
    fig.add_trace(go.Scatter(name='R for r={}'.format(values[i]), x=np.arange(len(l[i][3])), y=l[i][3]))
#    fig.add_trace(go.Scatter(name='Cases for r={}'.format(values[i]), x=np.arange(len(l[i][4])), y=l[i][4].cumsum()))

In [0]:
fig.show()

In [0]:
fig2.show()

In [0]:
fig = go.Figure(data=[
    go.Scatter(name='S', x=np.arange(len(model3[0])), y=model[0]),    
    go.Scatter(name='E', x=np.arange(len(model3[0])), y=model[1]),
    go.Scatter(name='I', x=np.arange(len(model3[0])), y=model[2]),
    go.Scatter(name='R', x=np.arange(len(model3[0])), y=model[3]),
    go.Scatter(name='Cases total', x=np.arange(len(model[0])), y=model[4].cumsum())
])

fig.show()

NameError: ignored

In [0]:
g=pd.DataFrame(gamma.rvs(4, loc=4, scale=1, size=100000), columns=['g'])

In [0]:
g.g.mean()

7.994704981275385

In [0]:
g.g.std()

2.000349092807803